In [ ]:
import xtdata as xt
data=xt.getStockDataAndDownload(stock_list=["000002.SZ","301611.SZ"],period="1d",start_time="19991205",end_time="20241105",fill_data=False,forceDownLoad=True,dividend_type="none")
# data=xtdata.get_market_data(field_list=[], stock_list=["301611.SZ"],period="1m",start_time="20221205",end_time="20231205",fill_data=False)
data["301611.SZ"]

In [ ]:
data["000002.SZ"]["20191220":"20241128"].to_csv("数据.csv")

In [ ]:
from xtquant import xtdata
import xtdata_utils
stocks = xtdata.get_stock_list_in_sector('沪深A股')  
data=xtdata_utils.getStockDataAndDownload(stock_list=stocks,period="1d",start_time="20150605",end_time="",fill_data=False,forceDownLoad=False,dividend_type="none")
# data=xtdata.get_market_data(field_list=[], stock_list=["301279.SZ"],period="1m",start_time="20230605",end_time="20240605",fill_data=False)
data

In [ ]:
from xtquant import xtdata
xtdata.download_history_data2(stock_list=stocks,
                    period="5m",start_time="20170101",end_time="",callback=lambda data:print(data),incrementally = True
                )


In [ ]:
from xtquant import xtdata
import xtdata_utils
stocks = xtdata.get_stock_list_in_sector('沪深A股')  
data=xtdata_utils.getStockDataAndDownload(stock_list=stocks,period="1d",start_time="20150605",end_time="",fill_data=False,forceDownLoad=False,dividend_type="none")
# data=xtdata.get_market_data(field_list=[], stock_list=["301279.SZ"],period="1m",start_time="20230605",end_time="20240605",fill_data=False)
data

In [ ]:
existing_data = xtdata.get_market_data(
                    field_list=["open", "high", "low", "close", "volume", "amount", "preClose"],stock_list=stocks,
                    period="1d",start_time="20150605",end_time="",fill_data=False,dividend_type="none"
                )


In [10]:
from pandas import DataFrame
import pandas as pd

def coverBatchStockToDataFrame(data:DataFrame):
    data={}
    for item in existing_data["close"].T.columns:
        result = pd.concat([
            existing_data["open"].loc[item], 
            existing_data["high"].loc[item],
            existing_data["close"].loc[item],
            existing_data["volume"].loc[item],
            existing_data["amount"].loc[item],
            existing_data["preClose"].loc[item]], axis=1,keys=["open", "high", "low", "close", "volume", "amount", "preClose"])
        data[item]=result
    return data

In [ ]:
stocks = xtdata.get_stock_list_in_sector('沪深A股')  
existing_data = xtdata.get_market_data(
                    field_list=["open", "high", "low", "close", "volume", "amount", "preClose"],stock_list=stocks,
                    period="1d",start_time="20150605",end_time="",fill_data=False,dividend_type="back"
                )
data=coverBatchStockToDataFrame(existing_data)
data

***** xtdata连接成功 *****
服务信息: None
服务地址: None
数据路径: None
设置xtdata.enable_hello = False可隐藏此消息



{'601882.SH':            open   high    low  close      volume  amount
 20150605    NaN    NaN    NaN      0         NaN     NaN
 20150608    NaN    NaN    NaN      0         NaN     NaN
 20150609    NaN    NaN    NaN      0         NaN     NaN
 20150610    NaN    NaN    NaN      0         NaN     NaN
 20150611    NaN    NaN    NaN      0         NaN     NaN
 ...         ...    ...    ...    ...         ...     ...
 20241125  21.96  22.25  21.92  26235  57413015.0   21.88
 20241126  21.85  21.96  21.49  32502  70240746.0   21.92
 20241127  21.49  22.36  22.30  38300  83079547.0   21.49
 20241128  22.17  22.28  21.81  24234  53260211.0   22.30
 20241129  22.01  22.87  22.45  40863  91165738.0   21.81
 
 [2307 rows x 6 columns],
 '603995.SH':            open   high    low  close      volume  amount
 20150605    NaN    NaN    NaN      0         NaN     NaN
 20150608    NaN    NaN    NaN      0         NaN     NaN
 20150609    NaN    NaN    NaN      0         NaN     NaN
 20150610    NaN  

5115

In [ ]:
def down(aa):
    print(aa)
    if(aa["finished"]==aa["total"]):
        print("结束了,开始展示")
        data=xtdata.get_market_data(field_list=[], stock_list=[aa["message"]],period="1m",start_time= "20201101",end_time='20201130',fill_data=False)
        print(data)

In [ ]:
from xtquant import xtdata
dd =xtdata.download_history_data2(stock_list=["000002.SZ"], period='1m',callback=down,start_time= "19991101",end_time='20241130', incrementally=True)
dd

In [ ]:
# 增量下载
from xtquant import xtdata
stocks = xtdata.get_stock_list_in_sector('沪深A股')  
#增量下载：
for index,s in enumerate(stocks):
    print(f"s ,{index}/{len(stocks)}")
    xtdata.download_history_data(stock_code=s, period='5m', start_time='',end_time='', incrementally=True)

In [ ]:
from xtquant import xtdata
stocks = xtdata.get_stock_list_in_sector('沪深A股')  
xtdata.download_history_data2(stock_list=stocks, period='1m', start_time='',end_time='', incrementally=True,callback=down1)

In [ ]:
from xtquant import xtdata
stocks = xtdata.get_stock_list_in_sector('沪深A股')  
xtdata.download_history_data2(stock_list=stocks, period='5m', start_time='',end_time='', incrementally=True,callback=down)

In [ ]:
xtdata.download_history_data2(stocks, '5m', '','', incrementally=True)
xtdata.download_history_data2(stocks, '15m', '','', incrementally=True)
xtdata.download_history_data2(stocks, '30m', '','', incrementally=True)

In [ ]:
from xtquant import xtdata
xtdata.get_market_data_ex(
    field_list=[],# 字段
    stock_list=["601318.SH"],# 合约代码列表
    period='1m',# 数据周期——1m、5m、1d、tick
    start_time='20240501',# 数据起始时间%Y%m%d或%Y%m%d%H%M%S
    end_time='20241101',# 数据结束时间%Y%m%d或%Y%m%d%H%M%S
    count=-1, # 数据个数
    dividend_type='none', # 除权方式
    fill_data=False, # 是否填充数据
)


In [ ]:

from xtquant import xtdata
import time


def my_download(stock_list:list,period:str,start_date = '', end_date = ''):
    '''
    用于显示下载进度
    '''
    import string
    
    if "d" in period:
        period = "1d"
    elif "m" in period:
        numb = period.translate(str.maketrans("", "", string.ascii_letters))
        if int(numb) < 5:
            period = "1m"
        else:
            period = "5m"
    elif "tick" == period:
        pass
    else:
        raise KeyboardInterrupt("周期传入错误")


    n = 1
    num = len(stock_list)
    for i in stock_list:
        print(f"当前正在下载 {period} {n}/{num}")
        
        xtdata.download_history_data(i,period,start_date, end_date)
        n += 1
    print("下载任务结束")

def do_subscribe_quote(stock_list:list, period:str):
  for i in stock_list:
    xtdata.subscribe_quote(i,period = period)
  time.sleep(1) # 等待订阅完成

if __name__ == "__main__":

  start_date = ''# 格式"YYYYMMDD"，开始下载的日期，date = ""时全量下载
  end_date = "" 
  period = "1m" 

  need_download = 1  # 取数据是空值时，将need_download赋值为1，确保正确下载了历史数据
  
  code_list = ["000001.SZ", "600519.SH"] # 股票列表

  if need_download: # 判断要不要下载数据, gmd系列函数都是从本地读取历史数据,从服务器订阅获取最新数据
    my_download(code_list, period=period, start_date, end_date)
  
  ############ 仅获取历史行情 #####################
  count = -1 # 设置count参数，使gmd_ex返回全部数据
  data1 = xtdata.get_market_data_ex([],code_list,period = period, start_time = start_date, end_time = end_date)

  ############ 仅获取最新行情 #####################
  do_subscribe_quote(code_list,period)# 设置订阅参数，使gmd_ex取到最新行情
  count = 1 # 设置count参数，使gmd_ex仅返回最新行情数据
  data2 = xtdata.get_market_data_ex([],code_list,period = period, start_time = start_date, end_time = end_date, count = 1) # count 设置为1，使返回值只包含最新行情

  ############ 获取历史行情+最新行情 #####################
  do_subscribe_quote(code_list,period) # 设置订阅参数，使gmd_ex取到最新行情
  count = -1 # 设置count参数，使gmd_ex返回全部数据
  data3 = xtdata.get_market_data_ex([],code_list,period = period, start_time = start_date, end_time = end_date, count = -1) # count 设置为1，使返回值只包含最新行情


  print(data1[code_list[0]].tail())# 行情数据查看
  print(data2[code_list[0]].tail())
  print(data3[code_list[0]].tail())





In [ ]:
# 用前须知

## xtdata提供和MiniQmt的交互接口，本质是和MiniQmt建立连接，由MiniQmt处理行情数据请求，再把结果回传返回到python层。使用的行情服务器以及能获取到的行情数据和MiniQmt是一致的，要检查数据或者切换连接时直接操作MiniQmt即可。

## 对于数据获取接口，使用时需要先确保MiniQmt已有所需要的数据，如果不足可以通过补充数据接口补充，再调用数据获取接口获取。

## 对于订阅接口，直接设置数据回调，数据到来时会由回调返回。订阅接收到的数据一般会保存下来，同种数据不需要再单独补充。

# 代码讲解

# 从本地python导入xtquant库，如果出现报错则说明安装失败
from xtquant import xtdata
import time
import pandas as pd
import backtrader as bt
%matplotlib inline

# 设定一个标的列表
code_list = ["000001.SZ"]
# 设定获取数据的周期
period = '5m'

# # 下载标的行情数据
# if 1:
#     ## 为了方便用户进行数据管理，xtquant的大部分历史数据都是以压缩形式存储在本地的
#     ## 比如行情数据，需要通过download_history_data下载，财务数据需要通过
#     ## 所以在取历史数据之前，我们需要调用数据下载接口，将数据下载到本地
#     for i in code_list:
#         xtdata.download_history_data(i,period=period,incrementally=True) # 增量下载行情数据（开高低收,等等）到本地
    
#     xtdata.download_financial_data(code_list) # 下载财务数据到本地
#     xtdata.download_sector_data() # 下载板块数据到本地
#     # 更多数据的下载方式可以通过数据字典查询

# # 读取本地历史行情数据
# history_data = xtdata.get_market_data_ex([],code_list,period=period,count=-1)
# print(history_data)
# print("=" * 20)

# 如果需要盘中的实时行情，需要向服务器进行订阅后才能获取
# 订阅后，get_market_data函数于get_market_data_ex函数将会自动拼接本地历史行情与服务器实时行情

# 向服务器订阅数据
for i in code_list:
    xtdata.subscribe_quote(i,period=period,count=-1) # 设置count = -1来取到当天所有实时行情

# 等待订阅完成
time.sleep(1)


# # 获取订阅后的行情
kline_data = xtdata.get_market_data_ex([],code_list,period=period,start_time='20240401')

# # 获取订阅后的行情，并以固定间隔进行刷新,预期会循环打印10次
# for i in range(10):
#     # 这边做演示，就用for来循环了，实际使用中可以用while True
#     kline_data = xtdata.get_market_data_ex([],code_list,period=period)
#     print(kline_data)
#     time.sleep(3) # 三秒后再次获取行情

# # 如果不想用固定间隔触发，可以以用订阅后的回调来执行
# # 这种模式下当订阅的callback回调函数将会异步的执行，每当订阅的标的tick发生变化更新，callback回调函数就会被调用一次
# # 本地已有的数据不会触发callback
    
# # 定义的回测函数
#     ## 回调函数中，data是本次触发回调的数据，只有一条
# def f(data):
#     # print(data)
    
#     code_list = list(data.keys())    # 获取到本次触发的标的代码

#     kline_in_callabck = xtdata.get_market_data_ex([],code_list,period = period)    # 在回调中获取klines数据
#     print(kline_in_callabck)

# for i in code_list:
#     xtdata.subscribe_quote(i,period=period,count=-1,callback=f) # 订阅时设定回调函数

# # 使用回调时，必须要同时使用xtdata.run()来阻塞程序，否则程序运行到最后一行就直接结束退出了。
# xtdata.run()

# 将time列转换为pandas的datetime类型
kline_data["000001.SZ"]['stime'] = pd.to_datetime(kline_data["000001.SZ"].index, format='%Y%m%d%H%M%S')

# 设置time列为索引
kline_data["000001.SZ"].set_index('stime', inplace=True)

print(kline_data["000001.SZ"])
kline_data["000001.SZ"].to_csv("000001.csv")
# 创建一个PandasData对象
datafeed = bt.feeds.PandasData(dataname=kline_data["000001.SZ"])

# 创建一个Cerebro引擎
cerebro = bt.Cerebro()
cerebro.adddata(datafeed)

# 运行策略
cerebro.run()

# 可视化结果
cerebro.plot()[0][0]

In [ ]:
kline_data["000001.SZ"].index

In [ ]:
import pandas as  pd
import tushare as ts
from datetime import datetime
import backtrader as bt
import matplotlib
import matplotlib.pyplot as plt
from matlab_utils.utils import set_chinese_font
plt.rcParams['figure.figsize'] = 15, 10
%matplotlib inline
set_chinese_font()
class gloVar:
    buy_count = 0
    sell_count = 0

class Bollstrategy(bt.Strategy):
    def log(self, txt, dt=None):
        dt = dt or self.datas[0].datetime.date(0)
        print('%s,%s' % (dt.isoformat(), txt))

    def __init__(self):
        print("进入__init__")
        # 指定价格序列
        self.dataclose = self.datas[0].close

        #交易订单状态初始化
        self.order = None

        #计算布林线
        ##使用自带的indicators中自带的函数计算出支撑线和压力线，period设置周期，默认是20
        self.lines.top = bt.indicators.BollingerBands(self.datas[0], period=20).top
        self.lines.bot = bt.indicators.BollingerBands(self.datas[0], period=20).bot

    def next(self):
        print("进入next")
        # 检查订单状态
        if self.order:
            print("等待成交")
            return

        # 检查持仓
        if not self.position:
            # 没有持仓，买入开仓
            if self.dataclose <= self.lines.bot[0]:
                print("================")
                print("收盘价跌破lower线，执行买入")
                self.order = self.buy()
        else:
            # 手里有持仓，判断卖平
            if self.dataclose >= self.lines.top[0]:
                print("======================")
                print("收盘价超过upper线，执行卖出")
                self.order = self.sell()

    def notify(self, order):

        if order.status in [order.Submitted, order.Accepted]:
            if order.status in [order.Submitted]:
                self.log("提交订单......")
            if order.status in [order.Accepted]:
                self.log("接受订单......")
            return

        # Check if an order has been completed
        # Attention: broker could reject order if not enougth cash
        if order.status in [order.Completed, order.Canceled, order.Margin]:
            if order.isbuy():
                self.log('执行买入, %.2f' % order.executed.price)
                gloVar.buy_count = gloVar.buy_count + 1
            elif order.issell():
                self.log('执行卖出, %.2f' % order.executed.price)
                gloVar.sell_count = gloVar.sell_count + 1
            self.bar_executed = len(self)

        self.log("订单完成......")
        print("======================")
        # Write down: no pending order
        self.order = None


token = '6aecc221a107b6aa227fb3d1765d85c0eeacfa2a4abf4d6d98b57023'
ts.set_token(token)
pro = ts.pro_api(timeout=60)
# 指定股票代码
ts_code = '000001.SZ'
code = '000001'
start_date = '20200101'
# 获取数据
def get_data(start):
    df = ts.pro_bar(ts_code=ts_code, adj='hfq', start_date=start)
    df = df.iloc[::-1]
    df.index = pd.to_datetime(df.trade_date)
    df['openinterest'] = 0
    df = df[['open', 'high', 'low', 'close', 'vol', 'openinterest']]
    df = df.rename(columns={'vol': 'volume'})
    return df

start_time = '2020-01-01'
accept_date ='2022-12-01'  # 查找这个日期是买入卖出点的股票

#加载数据
start = datetime.strptime(start_time, "%Y-%m-%d")
end = datetime.strptime(accept_date, "%Y-%m-%d")
buy_count = 0   # 买入次数
sell_count = 0   # 卖出次数
k_line_data = get_data(start.date().strftime("%Y%M%D"))
print(k_line_data)
k_line_data.to_csv("Quantative-trading-master\\csvfile\\" + ts_code + ".csv")
data = bt.feeds.PandasData(dataname=k_line_data, fromdate=start, todate=end)

# 加载backtrader引擎
back_trader = bt.Cerebro()
# 将数据传入
back_trader.adddata(data)
# 策略加进来
back_trader.addsizer(bt.sizers.FixedSize, stake=2000)
# 设置以收盘价成交，作弊模式
back_trader.broker.set_coc(True)
# 账号资金初始化
startCash = 100000
back_trader.broker.set_cash(startCash)
# 设置手续费
back_trader.broker.setcommission(commission=0.001)


# 布林线规则：跌破下轨，买入；超过上轨，卖出
back_trader.addstrategy(strategy=Bollstrategy)

# 输出初始化数据
d1 = start.date().strftime("%Y-%m-%d")
d2 = end.date().strftime("%Y-%m-%d")
print(f'初始化资金：{startCash},回测时间：{d1}：{d2}')
# 开启回测
result = back_trader.run()

# Print out the final result
print('最终资金: %.2f' % back_trader.broker.getvalue())
profit_ratio = (int(back_trader.broker.getvalue()) - startCash) / startCash * 100
print('投资收益率: %.2f%%' % profit_ratio)

print('买入次数：%s ；卖出次数：%s' % (gloVar.buy_count, gloVar.sell_count))
print("股票代码：%s" %ts_code)
# Plot the result
# back_trader.plot()
back_trader.plot(style='candlestick')[0][0]  # 画图

